In [296]:
from selenium import webdriver
from bs4 import BeautifulSoup
import pickle5 as pickle
import pandas as pd
import requests
import os.path
import re

In [310]:
# Extract page
def extract(URL):
    #headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/90.0.4430.212 Safari/537.36'}
    page = requests.get(URL)
    soup = BeautifulSoup(page.content, 'html.parser')
    
    #Download website locally ; Replace the path with where the folder is located on your system
    file_name = '/Users/ummerabab-/Desktop/wg-gesucht/Websites/' + URL.split("https://www.wg-gesucht.de/")[1].split('.html')[0] + '.txt'
    with open(file_name, "wb") as file:
        file.write(page.content)
    
    return soup

# Acquire useful information from HTML tags
def transform(soup):
    panel_body = soup.find('div', class_ = 'panel-body')
    rows = panel_body.find_all('div', class_ = 'row')
    
    details_dict = {}
    
    #WG-Details
    wg_tag = ''
    gesucht_wird = ''
    
    for item in rows:
        
        for sub_item in item.find_all('div', class_ = 'col-sm-6'):
            if sub_item.h4.text.strip() == 'Die WG':
                ul_tag = sub_item.find('ul')
                for li in ul_tag.find_all('li'):
                    wg_tag = wg_tag + ' ; ' + re.sub('\s+',' ',(li.text.strip().replace('\n','')))
                
            if sub_item.h4.text.strip() == 'Gesucht wird':
                ul_tag = sub_item.find('ul')
                for li in ul_tag.find_all('li'):
                    gesucht_wird = re.sub('\s+',' ',(li.text.strip().replace('\n','')))
    
    
    
    details_dict = {'Die WG': wg_tag,
                    'Gesucht wird': gesucht_wird}
        
    
    #Description
    description_tag = panel_body.find('div', id = 'ad_description_text')
    #print(description_tag)

    zimmer = ''
    lage = ''
    wg_leben = ''
    sonstiges = ''
        
    if description_tag.find('div', id ='freitext_0'):
            zimmer = description_tag.find('p', id ='freitext_0_content').text.strip().replace('\n','').replace('\r','')
    if description_tag.find('div', id ='freitext_1'):
            lage = description_tag.find('p', id ='freitext_1_content').text.strip().replace('\n','').replace('\r','')
    if description_tag.find('div', id ='freitext_2'):
            wg_leben = description_tag.find('p', id ='freitext_2_content').text.strip().replace('\n','').replace('\r','')
    if description_tag.find('div', id ='freitext_3'):
            sonstiges = description_tag.find('p', id ='freitext_3_content').text.strip().replace('\n','').replace('\r','')

            
    details_dict['Zimmer'] = zimmer
    details_dict['Lage'] = lage
    details_dict['WG-Leben'] = wg_leben
    details_dict['Sonstiges'] = sonstiges
        
    #print(desc_dict)
            
    return details_dict

In [320]:
#Specify the URL to extract from
URL = "https://www.wg-gesucht.de/wg-zimmer-in-Passau-Haidenhof-Sud.8691963.html"

# Call functions to Extract HTML from URL and Transform it.
page = extract(URL)
details_dict = transform(page)
print(details_dict)

{'Die WG': ' ; 22m² Zimmer in 5er WG ;  ; 5er WG ( 3 Frauen und 1 Mann ) ; Bewohneralter: 21 bis 27 Jahre ; Studenten-WG, keine Zweck-WG, gemischte WG, Vegetarisch/Vegan, LGBTQIA+ ; ', 'Gesucht wird': 'Geschlecht egal zwischen 20 und 27 Jahren', 'Zimmer': "Dein Zimmer ist - nach Max' Meinung ;) - das schönste in diesem Traum eines studentischen Domizils.Dein Zimmer hat eine Tür zum Balkon mit Blick auf die Berge Inglings und Österreichs mit Sonnenliege und einer bezaubernden Stimmung bei Nacht. Es ist absolut still: Lediglich die vielfältige Vogel- und Tierwelt (wir haben auch Igel) sowie die herzlichen Gespräche aus den benachbarten Gärten oder das Klingeln des Postboten dringen zu dir vor.Das Zimmer selbst ist mit einem Schreibtisch, einer Kommode und einem Schrank ausgestattet -  lediglich das Bett, den gelben Sessel und die Lampe nimmt Max mit. Die breite Fensterfront mit den altmodischen Fensterläden gibt einen herrlichen Blick ins Grüne und auf das Inntal preis. Du besitzt einen 

In [321]:
# Convert dictionary to data_frame
# data_items = details_dict.items()
# data_list = list(data_items)
data_frame = pd.DataFrame([details_dict])
print (data_frame)

# Save to file with pickle in csv format
# data_frame.to_pickle('descriptions')
csv_path = '/Users/ummerabab-/Desktop/wg-gesucht/Code/dataset.csv'
if(os.path.isfile(csv_path)):
    data_frame.to_csv(csv_path, mode='a', encoding='latin1', index= False,header=False)
else:
    data_frame.to_csv(csv_path, encoding='latin1', index= False)
#data_frame.to_csv(r'/Users/ummerabab-/Desktop/wg-gesucht/Code/dataset.csv', index = False)

                                              Die WG  \
0   ; 22m² Zimmer in 5er WG ;  ; 5er WG ( 3 Fraue...   

                                Gesucht wird  \
0  Geschlecht egal zwischen 20 und 27 Jahren   

                                              Zimmer  \
0  Dein Zimmer ist - nach Max' Meinung ;) - das s...   

                                                Lage  \
0  Die ehemalige Stadtvilla thront idyllisch hint...   

                                            WG-Leben  \
0  Diese WG scheint immer wieder Menschen anzuzie...   

                                      Sonstiges  
0  Wir freuen uns über schriftliche Anfragen :)  


In [317]:
# driver = webdriver.Chrome()
# driver.get("https://www.wg-gesucht.de/wg-zimmer-in-Passau.104.0.1.0.html")
# print(driver.title)

In [ ]:
# def transform(driver):
#     #/html/body/div[@id='master_wrapper']/div[@id='site_wrapper']/div[@id='main_content']/div[@id='main_column']/div[@id='liste-details-ad-7368387']/div[@class='row']/div[@class='col-sm-8 card_body']
#     cards = driver.find_element_by_xpath("/html/body/div[@id='master_wrapper']/div[@id='site_wrapper']/div[@id='main_content']/div[@id='main_column']/div[@id='liste-details-ad-7368387']")
#     print(cards)
#     cards.click()
#     #return len(divs)

# transform(driver)

In [ ]:
#     h3 = []
#     h4 = []
#    for item in rows:
        #print('This is an item' + str(item))
#         try:
#             #h3
#             if item.find('div', class_ = 'col-xs-12'):
#                 #h3.append(item.find('h3', class_ = 'headline headline-detailed-view-panel-title').text.strip())
#                 h3.append(item.h3.text.strip())
#                 #print(h3)
#             #h4
#             for col in item.find_all('div', class_ = 'col-sm-6'):
#                 h4.append(h4.append(col.h4.text.strip()))

#                 #h4.append(item.find_all('h4', class_ = 'headline headline-detailed-view-datasheet').text.strip())
#                 #print(h4)

#             description = item.find('p', class_ = 'freitext').text.strip().replace('\n\r\n', '')
#         except:
#             description = ''
        

#         wg_list.append(h3)
#         wg_list.append(h4)